In [2]:
from squircle import Squircle, deserialize

220nm og 250nm
440nm og 500nm
320nm og 380nm
220nm og 250nm
160nm og 190nm
100nm og 130nm

440nm og 500nm
320nm og 380nm
200nm og 260nm
200nm og 260nm
220nm og 250nm
220nm og 250nm
160nm og 190nm
100nm og 130nm
100nm og 130nm


squircle1 = deserialize("data/magnetic-data/002-00")

Squircle(figure_name='002', index=0, squircle_factor=0, data=array([[[ 1.01953403, -0.68660657],
        [ 0.7772757 , -0.98541905],
        [ 0.39687869, -1.16220027],
        ...,
        [ 1.55031816,  0.55462664],
        [ 1.62924614,  0.48863763],
        [ 1.22640847,  0.02790944]],

       [[ 0.89156809, -1.47867864],
        [ 0.26518894, -1.57191598],
        [ 0.30552298, -1.66165498],
        ...,
        [ 0.84729941, -0.04739771],
        [ 0.91066108, -0.06770606],
        [ 0.92327469, -0.07963284]],

       [[ 0.88603167, -1.72217215],
        [ 0.61108799, -1.73464684],
        [ 0.11724924, -1.86016803],
        ...,
        [ 0.30782032, -0.84830287],
        [ 0.58777915, -0.4595273 ],
        [ 0.54848524, -0.18989331]],

       ...,

       [[-0.354833  , -0.81152837],
        [ 0.11421229, -0.81729614],
        [ 0.038634  , -0.59042478],
        ...,
        [-0.76394878, -1.89285385],
        [ 0.16220064, -1.92551203],
        [ 1.04447378, -1.20284483]],

  